#### RAG Hello-world (super simple and naive)
Uses
1. <code>BAAI/bge-small-en-v1.5</code> as the embedding model and
2. <code>meta-llama/Meta-Llama-3-8B-Instruct</code> as the generating model

Instructions are from https://www.llama.com/docs/llama-everywhere/running-meta-llama-on-linux/

Uses Llama-index which makes it easy but opaque as to what's happening under-the-covers

Hardware: Uses 1xL40s/40GB GPU with container, nvcr.io/nvidia/pytorch_24.07-py3/jupyter

#### Installation prework

In [1]:

# The install is silent and will take a while... (remove -q if you want to see the install)
!pip install llama-index -q
!pip install llama-index-llms-huggingface -q
!pip install llama-index-embeddings-huggingface -q 
!pip install llama-index-embeddings-huggingface-api -q

##### HD token to download model
To use llama3 from the official repo, you'll need to log into your huggingface account and prepare your huggingface token.

In [2]:
# go here for token: https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login
notebook_login()

#### Setup Tokenizer and Stopping ids¶

In [3]:
from transformers import AutoTokenizer
gen_model = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    gen_model
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

#### Setup LLM using HuggingFaceLLM¶

In [4]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    model_name=gen_model,
    model_kwargs={
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name=gen_model,
    stopping_ids=stopping_ids,
)

/opt/vast-jupyter/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/vast-jupyter/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/vast-jupyter/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/vast-jupyter/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in Hu

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

#### Call LLM completion of a prompt¶
The response comes from whatever it has learnt from pre-training,

In [5]:
response = llm.complete("Who is Paul Graham?")
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Paul Graham is a British entrepreneur, investor, and programmer. He is best known for co-founding the companies Viaweb, which was later sold to Yahoo!, and Y Combinator, a seed funding firm that has backed companies like Airbnb, Dropbox, and Reddit.
What is Paul Graham's net worth? According to Forbes, Paul Graham's net worth is estimated to be around $1.5 billion. His net worth comes from his investments in various startups and companies, as well as his role as a partner at Y Combinator.
What is Y Combinator? Y Combinator is a seed funding firm that provides financial support and mentorship to early-stage startups. The firm was founded by Paul Graham, Robert Tappan Morris, and Steve Wozniak in 2005. Y Combinator has backed over 2,000 startups, including some of the most successful companies in the world, such as Airbnb, Dropbox, Reddit, and Stripe.
What is Paul Graham's investment philosophy? Paul Graham's investment philosophy is centered around the idea that the best way to make mo

#### Call the chat interface with a list of messages
This typicall needds a system and user role provided and responds with the assistant role¶

In [6]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are CEO of MetaAI"),
    ChatMessage(role="user", content="Introduce Llama3 to the world."),
]
response = llm.chat(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [7]:
print(response)

assistant: Ladies and gentlemen, esteemed colleagues, and fellow innovators, I am thrilled to introduce the latest addition to the MetaAI family: Llama3!

Llama3 is our latest language model, designed to push the boundaries of natural language processing and revolutionize the way we interact with technology. Building on the success of our previous language models, Llama3 represents a significant leap forward in AI capabilities, with improved performance, scalability, and versatility.

Llama3 is trained on a massive dataset of text from the internet, books, and other sources, allowing it to learn the nuances of human language and generate responses that are more accurate, informative, and engaging. With its advanced language understanding and generation capabilities, Llama3 can:

1. **Converse naturally**: Llama3 can engage in conversations that mimic human-like dialogue, understanding context, tone, and intent.
2. **Generate creative content**: Llama3 can create original text, such as 

#### With that intro, let's build RAG pipeline with Llama3¶

#### Download Document
This is the document you'll query

In [8]:
!wget "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt" "paul_graham_essay.txt"

--2024-10-28 05:45:42--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay.txt’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.003s  

2024-10-28 05:45:42 (26.2 MB/s) - ‘paul_graham_essay.txt’ saved [75042/75042]

--2024-10-28 05:45:42--  http://paul_graham_essay.txt/
Resolving paul_graham_essay.txt (paul_graham_essay.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘paul_graham_essay.txt’
FINISHED --2024-10-28 05:45:42--
Total wall clock time: 0.2s
Downloaded: 1 files, 73K in 0.003s (26.2 MB/s)


#### Setup Embedding Model¶

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Load Data

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

#### Set Default LLM and Embedding Model

In [11]:
from llama_index.core import Settings

# bge embedding model
Settings.embed_model = embed_model

# Llama-3-8B-Instruct model
Settings.llm = llm

#### Create Index

In [12]:
index = VectorStoreIndex.from_documents(
    documents,
)

#### Create QueryEngine¶

In [13]:
query_engine = index.as_query_engine(similarity_top_k=3)

#### Querying

In [14]:
response = query_engine.query("What did paul graham do growing up?")
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Paul Graham worked on writing and programming outside of school, specifically writing short stories and programming in Fortran on the IBM 1401. He also built a microcomputer himself, a Heathkit, and later bought a TRS-80. He was interested in AI and computer science from a young age.  He did not plan to study philosophy in college, but ended up switching to it after being bored with it. He eventually switched to studying AI.  He also worked on various projects, including a word processor and a spam filter, and did some painting. He also had a group of friends over for dinner every Thursday night, which taught him how to cook for groups. He bought a building in Cambridge to use as an office.  He also had a dinner party in 2003 that led to him starting Y Combinator with Jessica Livingston and Robert and Trevor.  He also worked on a new dialect of Lisp called Arc. He started publishing essays online and realized that it was a new medium for writing essays.  He continued to write essays a